In [ ]:
!pip install PyMuPDF
!pip install pymongo
!pip install PyDrive2
!pip install OpenAI
!pip install pytesseract
!pip install bert-score
!pip install rouge-score
!sudo apt install tesseract-ocr tesseract-ocr-deu

In [ ]:
!pip install langchain langchain_community langchain-openai

In [ ]:
import fitz  # PyMuPDF
from PIL import Image
from google.colab import drive

In [ ]:
drive.mount('/content/drive/', force_remount=True)

In [ ]:
def pdf_to_images(pdf_path, output_folder):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    number_of_pages = pdf_document.page_count

    # Loop through each page
    for page_num in range(number_of_pages):
        # Get the page
        page = pdf_document.load_page(page_num)
        # Render page to an image
        pix = page.get_pixmap(dpi=200)

        # Convert to a PIL image
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        # Save the image
        img.save(f"{output_folder}/page_{page_num + 1}.png")

In [ ]:
pdf_to_images("/content/drive/MyDrive/bachelor-thesis/pdfs/01.pdf", "/content/drive/MyDrive/bachelor-thesis/output-images")

In [ ]:
!gcloud init

In [ ]:
!gcloud auth application-default login

In [ ]:
google_token = !gcloud auth print-access-token
google_token = google_token[0]

In [ ]:
import requests
import base64
import numpy

In [ ]:
def get_image_text_embedding(image_url, text):

  b64img = base64.b64encode(requests.get(image_url).content).decode("ascii")

  google_token = !gcloud auth print-access-token
  google_token = google_token[0]

  text = text[:1000]

  response = requests.post(
      headers={
          "Authorization": "Bearer " + google_token,
          "Content-Type": "application/json"
          },
      url="https://europe-west3-aiplatform.googleapis.com/v1/projects/bachelor-thesis-428711/locations/europe-west3/publishers/google/models/multimodalembedding@001:predict",
      json={
        "instances": [
          {
            "image": {
              "bytesBase64Encoded": b64img
            },
            "text": text
          }
        ],
        "parameters": {
          "dimension": 1408
        }
      }
  )

  return (numpy.array(response.json()['predictions'][0]['imageEmbedding']) * 0.5 + numpy.array(response.json()['predictions'][0]['textEmbedding']) * 0.5).tolist()

In [ ]:
def get_text_embedding(text):
  text = text[:1000]

  google_token = !gcloud auth print-access-token
  google_token = google_token[0]

  response = requests.post(
      headers={
          "Authorization": "Bearer " + google_token,
          "Content-Type": "application/json"
          },
      url="https://europe-west3-aiplatform.googleapis.com/v1/projects/bachelor-thesis-428711/locations/europe-west3/publishers/google/models/multimodalembedding@001:predict",
      json={
        "instances": [
          {
            "text": text
          }
        ],
        "parameters": {
          "dimension": 1408
        }
      }
  )

  return response.json()['predictions'][0]['textEmbedding']

In [ ]:
import pytesseract
import requests
import cv2
import numpy as np
from PIL import Image
import io
from matplotlib import pyplot as plt

def recognize_text(imgurl):
  response = requests.get(imgurl)
  img = Image.open(io.BytesIO(response.content))

  img_array = np.asarray(img)

  kernelSize = 5
  maxKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernelSize, kernelSize))
  localMax = cv2.morphologyEx(img_array, cv2.MORPH_CLOSE, maxKernel, None, None, 1, cv2.BORDER_REFLECT101)

  gainDivision = np.where(localMax == 0, 0, (img_array/localMax))

  gainDivision = np.clip((255 * gainDivision), 0, 255)

  gainDivision = gainDivision.astype("uint8")

  grayscaleImage = cv2.cvtColor(gainDivision, cv2.COLOR_BGR2GRAY)

  _, binaryImage = cv2.threshold(grayscaleImage, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

  text = pytesseract.image_to_string(binaryImage, lang="eng+deu")
  return text

In [ ]:
import pymongo
import requests
from google.colab import userdata

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
mongo_client = pymongo.MongoClient(userdata.get('CLUSTER_BACHELOR_CLUSTER_CONNECTION_STRING'))

db = mongo_client["approach-2-new"]
collection = db["embeddings"]

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
output_images_folder = drive.ListFile({'q': "mimeType = 'application/vnd.google-apps.folder' and title = 'output-images'" }).GetList()[0]

output_images_files = drive.ListFile({'q': "'"+ output_images_folder['id'] + "' in parents and fileExtension = 'png'"}).GetList()

In [ ]:
data_list = []

for file in output_images_files:
  '''file.InsertPermission({
      'type': 'anyone',
      'value': 'anyone',
      'role': 'reader'
  })'''
  image_url = "https://drive.usercontent.google.com/download?id=" + file['id'] + "&authuser=0"
  response = requests.get(image_url)

  image_text = recognize_text(image_url)
  total_embedding = get_image_text_embedding(image_url, image_text)


  data_list.append({"embedding": total_embedding, "url": image_url, "text": image_text})

In [ ]:
collection.insert_many(data_list)

Retrieve image for text query (similarity search):

In [ ]:
query = "What are the ESG targets?"

embedded_query = get_text_embedding(query)
len(embedded_query)

In [ ]:
def get_similar_images(query):
  embedded_query = get_text_embedding(query)

  similarity_search_pipeline = [
    {
      '$vectorSearch': {
        'index': 'approach_2_index',
        'path': 'embedding',
        'queryVector': embedded_query,
        'numCandidates': 150,
        'limit': 5
      }
    },
    {
      "$project": {
        "_id": 0,
        "embedding": 1,
        "url": 1,
        "text": 1,
        "score": { "$meta": "vectorSearchScore" }
      }
    }
  ]

  result = db.embeddings.aggregate(similarity_search_pipeline)

  image_list = []
  for image in result:
    image_list.append((image['url'], image['text']))

  return image_list

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import MessagesPlaceholder
from langchain_core.messages.system import SystemMessage
from langchain_core.messages.human import HumanMessage

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4o")

def get_answer(query):
  similar_images = get_similar_images(query)
  print(similar_images)

  image_prompt_text = []
  for url, text in similar_images:
    b64img = base64.b64encode(requests.get(url).content).decode("ascii")
    image_prompt_text.append({"type": "image_url", "image_url": {"url": f"data:image/png;base64,{b64img}"}})

  inner_text = f"\n\nThe following text is in the images: \n"
  for url, text in similar_images:
    inner_text += f"{text}\n\n"
  inner_text = inner_text.replace("{", "{{").replace("}", "}}")

  image_prompt_text.append({"type": "text", "text": "Based on the information in the images and the text in the images, answer the question: {question}" + inner_text})

  prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a powerful visual assistant who can answer questions based on information in the given images."),
    ("human", image_prompt_text)
  ])


  llm_chain = prompt | llm
  result = llm_chain.invoke(query).content

  return result

In [ ]:

import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from bert_score import score

def create_scores():
  df_bleu = pd.read_csv("/content/drive/MyDrive/bachelor-thesis/results/bleu.csv")
  df_rouge1 = pd.read_csv("/content/drive/MyDrive/bachelor-thesis/results/rouge-1.csv")
  df_rouge2 = pd.read_csv("/content/drive/MyDrive/bachelor-thesis/results/rouge-2.csv")
  df_rougel = pd.read_csv("/content/drive/MyDrive/bachelor-thesis/results/rouge-l.csv")
  df_bertscore = pd.read_csv("/content/drive/MyDrive/bachelor-thesis/results/bertscore.csv")

  for i, question in enumerate(df_bleu["question"]):
    if i >= 50:
      break

    while(True):
      print("Question " + str(i))
      try:
        answer = get_answer(question)
      except Exception as error:
        print("Error in getting answer. Trying again. Error:")
        print(error)
        continue
      break

    df_bleu["approach-2-text"][i] = answer
    df_rouge1["approach-2-text"][i] = answer
    df_rouge2["approach-2-text"][i] = answer
    df_rougel["approach-2-text"][i] = answer
    df_bertscore["approach-2-text"][i] = answer

    split_answer = answer.split()

    reference_answer = df_bleu["reference-text"][i]
    split_reference_answer = reference_answer.split()

    bleu_score = sentence_bleu([split_reference_answer], split_answer)

    _rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeLsum'], use_stemmer=True)
    rouge_scores = _rouge_scorer.score(reference_answer, answer)

    rouge1 = rouge_scores['rouge1'].precision
    rouge2 = rouge_scores['rouge2'].precision
    rougeLsum = rouge_scores['rougeLsum'].precision

    bertscore_tensor, _, _ = score(cands=[answer], refs=[reference_answer], lang="en")
    bertscore = bertscore_tensor.numpy()[0]


    df_bleu["approach-2-score"][i] = bleu_score
    df_rouge1['approach-2-score'][i] = rouge1
    df_rouge2['approach-2-score'][i] = rouge2
    df_rougel['approach-2-score'][i] = rougeLsum
    df_bertscore['approach-2-score'][i] = bertscore

  df_bleu.to_csv("/content/drive/MyDrive/bachelor-thesis/results/bleu.csv")
  df_rouge1.to_csv("/content/drive/MyDrive/bachelor-thesis/results/rouge-1.csv")
  df_rouge2.to_csv("/content/drive/MyDrive/bachelor-thesis/results/rouge-2.csv")
  df_rougel.to_csv("/content/drive/MyDrive/bachelor-thesis/results/rouge-l.csv")
  df_bertscore.to_csv("/content/drive/MyDrive/bachelor-thesis/results/bertscore.csv")

In [ ]:
create_scores()